In [37]:
import netsquid as ns
from netsquid.nodes import Node, Network
from netsquid.components import QuantumChannel, ClassicalChannel, QuantumProcessor
from netsquid.qubits import qubitapi as qapi
from netsquid.qubits.qubitapi import create_qubits, operate, reduced_dm, measure
from netsquid.qubits.qformalism import DenseDMRepr
from sympy import Matrix
from sympy.printing.latex import latex
from IPython.display import display, Math

import socket, pickle
import numpy as np

In [26]:
def apply_corrections(node, qubit_index, m1, m2):
    if m1 == 1:
        qapi.operate(node.qmemory.peek(qubit_index)[0], ns.X)
    if m2 == 1:
        qapi.operate(node.qmemory.peek(qubit_index)[0], ns.Z)

# Function to print the density matrix of the qubits in a node's quantum memory
def print_density_matrix(node, qubit_indices):
    qubits = [node.qmemory.peek(i)[0] for i in qubit_indices]
    dm = reduced_dm(qubits)
    dm_matrix = Matrix(dm)
    print(f"Density matrix for {node.name}:")
    display(Math(latex(dm_matrix)))
    return dm

def print_density_matrix_whole_system(nodes):
    qubits = []
    for node in nodes:
        for i in range(node.qmemory.num_positions):
            qubit = node.qmemory.peek(i)[0]
            if qubit is not None:
                qubits.append(qubit)
    dm = reduced_dm(qubits)
    dm_matrix = Matrix(dm)
    print("Density matrix for the whole system:")
    display(Math(latex(dm_matrix)))
    return dm

In [27]:
# Create nodes for Alice, Charlie, and Bob with enough quantum memory slots
alice = Node("Alice", qmemory=QuantumProcessor("AliceProcessor", num_positions=1))
bob = Node("Bob", qmemory=QuantumProcessor("BobProcessor", num_positions=1))

In [28]:

# Create a socket object 
s = socket.socket()         
 
# Define the port on which you want to connect 
port = 12346
host = 'localhost'            
 
# connect to the server on local computer 
s.connect((host, port)) 
 
# receive data from the server and decoding to get the string.
packet = s.recv(2048)
unpickled = pickle.loads(packet)
alice_state, bob_state = unpickled[0]
m1, m2 = unpickled[1]

# close the connection 
s.close()     

In [40]:
# Create qubits for Alice and Bob
alice_qubits = create_qubits(1, no_state=True)
bob_qubits = create_qubits(1, no_state=True)

# Set the state of the qubits to the density matrices stored in alice and bob
qapi.assign_qstate(alice_qubits[0], alice_state, formalism=DenseDMRepr)
qapi.assign_qstate(bob_qubits[0], bob_state, formalism=DenseDMRepr)

print_density_matrix(bob, [0])

# Add the qubits to each quantum memory
alice.qmemory.put(alice_qubits, positions=[0])
bob.qmemory.put(bob_qubits, positions=[0])

Density matrix for Bob:


<IPython.core.display.Math object>

In [41]:
# Apply corrections to Alice's and Bob's qubits
apply_corrections(alice, 0, m1, m2)
apply_corrections(bob, 0, m1, m2)

print_density_matrix(alice, [0])
print_density_matrix(bob, [0])
print_density_matrix_whole_system([alice, bob])

Density matrix for Alice:


<IPython.core.display.Math object>

Density matrix for Bob:


<IPython.core.display.Math object>

Density matrix for the whole system:


<IPython.core.display.Math object>

array([[0.25+0.j, 0.  +0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.25+0.j, 0.  +0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.25+0.j, 0.  +0.j],
       [0.  +0.j, 0.  +0.j, 0.  +0.j, 0.25+0.j]])